 # Stokes Inversion for GST/NIRIS Using Stacked Deep Neural Networks

### Haodi Jiang et al.

New Jersey Institute of Technology



## 1. Introduction

Obtaining high-quality magnetic and velocity fields through Stokes inversion is crucial in solar physics.
Here, we present a new deep learning method, named Stacked Deep Neural Networks (SDNN),
for inferring line-of-sight (LOS) velocities, Doppler widths and vector magnetic fields from Stokes profiles
collected by the Near InfraRed Imaging Spectropolarimeter (NIRIS) on the 1.6 m Goode Solar Telescope (GST)
at the Big Bear Solar Observatory (BBSO).

In this notebook, we provide an overview of the SDNN project,
detailing how it can be used to perform Stokes inversion for GST/NIRIS data.

Since model training requires GPUs, it is omitted here.


## 2. Workflow of SDNN


### 2.1 Load model


Note that the SDDN has large model and fits files that can not be saved in public repository. Therefore, they must be downloaded first.

In [1]:
from utils import *
download_model()
download_fits()


Finished dowloading the file.
Finished dowloading the file.


Import the load_model() function from SDNN module.


In [1]:
from SDNN import load_model
model = load_model()

Loading SDNN model...
Done Loading...


### 2.2 Setup input and output directory

You may set up your own data directory.

In [2]:
input_path = '/Users/sdpathak/Documents/njit/research/SDNN/inputs/'  # edit your input path
output_path = '/Users/sdpathak/Documents/njit/research/SDNN/outputs'  # edit your output path

#### 2.3 Predicting with Pretrained model

Import related libraries.

Produce LOS velocities, Doppler widths, and vector magnetic fields by using the pretrained model.

The code will also save b_total, inclination and azimuth if save_mag_field_o = True; otherwise it will only save bx, by, bz, Doppler width and LOS velocity.

The predicted results will be saved in the given output path.

In [2]:
# from astropy.io import fits
# import os
# import time
# import numpy as np
# from SDNN import read_data
# from SDNN import inverse
# from SDNN import save_results
# from SDNN import load_model

# model = load_model()

# for file in os.listdir(input_path):
#     print('---------- working on', file, '----------')
#     fits_file = os.path.join(input_path, file)
#     hdu = fits.open(fits_file)
#     hdu.verify('fix')
#     data = hdu[0].data
#     test_data, data_height, data_width = read_data(data)
#     start = time.time()
#     predict_results = inverse(test_data, model)
#     end = time.time()
#     print('Inversion Time:', np.round(end - start, 1), 's')
#     save_results(predict_results, output_path, file[: file.find('.')], data_height, data_width, save_mag_field_o=False)

from SDNN import load_model, read_data, inverse, save_results
from tensorflow.keras import backend as K
import numpy as np
import os
import time
from astropy.io import fits

# Define input and output paths
input_path = '/Users/sdpathak/Documents/njit/research/SDNN/inputs/'
output_path = '/Users/sdpathak/Documents/njit/research/SDNN/outputs/'

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Load the model
print("Loading the model...")
model = load_model()

# Process each input file
for file in os.listdir(input_path):
    # if not file.endswith('.fts'):
    #     print(f"Skipping non-FITS file: {file}")
    #     continue

    fits_file = os.path.join(input_path, file)
    
    try:
        print(f"---------- Working on {file} ----------")
        
        # Open and verify the FITS file
        hdu = fits.open(fits_file)
        hdu.verify('fix')
        data = hdu[0].data

        # Preprocess the data
        test_data, data_height, data_width = read_data(data)
        
        # Ensure test_data shape matches model input shape
        print(f"Test data shape before reshape: {test_data.shape}")
        
        # Reshape test_data to match model input
        test_data = np.reshape(test_data, (-1, 60, 4))  # Assuming model expects (batch_size, 60, 4)
        print(f"Test data shape after reshape: {test_data.shape}")
        print(f"Model input shape: {model.input_shape}")

        # Perform prediction
        start = time.time()
        predict_results = inverse(test_data, model)
        end = time.time()
        print(f"Inversion Time: {np.round(end - start, 1)} s")

        # Save the results
        save_results(
            predict_results, 
            output_path, 
            file[: file.find('.')],  # File name without extension
            data_height, 
            data_width, 
            save_mag_field_o=False
        )
    except Exception as e:
        print(f"Error processing file {file}: {e}")




Loading the model...
Loading SDNN model...
Done Loading...
---------- Working on cals_170713_162012.fts ----------
Loading test data...
Done loading...
Test data shape before reshape: (518400, 60, 4)
Test data shape after reshape: (518400, 60, 4)
Model input shape: (None, 60, 4)
Start inversion...
16200/16200 ━━━━━━━━━━━━━━━━━━━━ 917s 57ms/step
End inversion...
Inversion Time: 919.7 s
Producing inversion results..
Done saving..
---------- Working on .gitignore ----------
Error processing file .gitignore: Empty or corrupt FITS file


## 3. Conclusion

We develop a deep learning model (SDNN), which is an effective and alternative method for
inferring LOS velocities, Doppler widths, and vector magnetic fields from Stokes profiles of GST/NIRIS.
It is hoped that SDNN will be a useful tool in producing the high-quality velocity and
magnetic fields that are crucial for understanding the evolution of physical properties of the solar atmosphere.

## Acknowledgments

The authors gratefully acknowledge the use of data from the Goode Solar Telescope (GST) of the Big Bear Solar Observatory (BBSO).
The BBSO operation is supported by NJIT and U.S. NSF grant AGS-1821294.
This work was supported by U.S. NSF grants AGS-1927578, AGS-1954737, and AGS-2228996.


## References

Inferring Line-of-sight Velocities and Doppler Widths from Stokes Profiles of GST/
NIRIS Using Stacked Deep Neural Networks. Haodi Jiang et al 2022 ApJ 939 66

https://iopscience.iop.org/article/10.3847/1538-4357/ac927e